# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("./Orders.csv")

display(orders.head())

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# SUB PROBLEM 1
best_customers = orders.groupby("CustomerID")["amount_spent"].sum().sort_values(ascending=False)
display(best_customers.head())

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
Name: amount_spent, dtype: float64

In [4]:
# SUB PROBLEM 2
display(best_customers[(best_customers > best_customers.quantile(.1)) \
                       & (best_customers < best_customers.quantile(.5))].head())

CustomerID
14504    673.26
18121    673.10
14628    671.96
17883    671.77
12638    671.69
Name: amount_spent, dtype: float64

In [5]:
# SUB PROBLEM 3
# Vamos a otorgar el 2 a los VIP, el 1 a los Preferred y el 0 a los demás.

customers = pd.DataFrame()
customers["amount_spent"] = best_customers
customers["customersLabel"] = customers["amount_spent"].apply(lambda x: 2 if x > best_customers.quantile(.95)\
                                                              else 1 if x > best_customers.quantile(.75) else 0)

display(best_customers.head())

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
Name: amount_spent, dtype: float64

In [6]:
display(customers["customersLabel"].value_counts())

0    3254
1     868
2     217
Name: customersLabel, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
country_df = customers.merge(orders[["CustomerID", "Country"]], left_on="CustomerID", right_on="CustomerID",\
                             how= "left").drop_duplicates()
display(country_df.shape)
display(country_df[country_df["CustomerID"]==12455])

# Hay países con más de un país

(4347, 4)

,CustomerID,amount_spent,customersLabel,Country
211225,12455,2466.86,1,Cyprus
211267,12455,2466.86,1,Spain


In [8]:
# display(country_df.groupby("Country").sum())

display(country_df[country_df["customersLabel"] == 2].groupby("Country")["customersLabel"].count()\
        .sort_values(ascending=False).head(1))

Country
United Kingdom    177
Name: customersLabel, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
display(country_df[(country_df["customersLabel"] == 2) | (country_df["customersLabel"] == 1)]\
        .groupby("Country")["customersLabel"].count().sort_values(ascending=False).head(1))

Country
United Kingdom    932
Name: customersLabel, dtype: int64